## **Customer Churn Prediction - Logistic Regression Modeling**

This is a notebook with the notes taken from the [Machine Learning Zoomcamp](https://github.com/alexeygrigorev/mlbookcamp-code/tree/master/course-zoomcamp), based on the book **"Machine Learning Bookcamp"** written by Alexey Grigorev.

In [51]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import display

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Magic function to display matplotlib figures in a jupyter notebook
%matplotlib inline

In [2]:
data = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
len(data)

7043

In [3]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


## **Data Pre-processing**

In [5]:
# Converting TotalCharges column values to numeric
total_charges = pd.to_numeric(data["TotalCharges"], errors="coerce")    # coerce to force convertion of strings " " into NaN
data[total_charges.isnull()][["customerID", "TotalCharges"]]

,customerID,TotalCharges
488,4472-LVYGI,
753,3115-CZMZD,
936,5709-LVOEQ,
1082,4367-NUYAO,
1340,1371-DWPAZ,
3331,7644-OMVMY,
3826,3213-VVOLG,
4380,2520-SGTTA,
5218,2923-ARZLG,
6670,4075-WKNIU,


In [6]:
# Setting missing values to zero
data["TotalCharges"] = pd.to_numeric(data["TotalCharges"], errors="coerce")
data["TotalCharges"] = data["TotalCharges"].fillna(0)

### Standarizing column names and values

In [7]:
data.columns = data.columns.str.lower().str.replace(" ", "_")
data.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [8]:
string_columns = list(data.dtypes[data.dtypes == "object"].index)
for col in string_columns:
    data[col] = data[col].str.lower().str.replace(" ", "_")

data.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [9]:
# Looking into target variable
data["churn"].head()

0     no
1     no
2    yes
3     no
4    yes
Name: churn, dtype: object

In [ ]:
data["churn"].unique()

In [10]:
# Changing yes/no for 1/0
data["churn"] = data["churn"].apply(lambda x: 1 if x == "yes" else 0)
data["churn"]

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: churn, Length: 7043, dtype: int64

In [11]:
# Splitting data into training and testing subsets
df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=1)

In [12]:
df_train_full.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
1814,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
5946,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
3881,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
2389,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
3676,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [13]:
# Obtaining the validation subset
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

# Defining target variables for each subset
y_train = df_train["churn"].values
y_val = df_val["churn"].values

# Deleting the target variables from subsets to avoid perfect fitting
del df_train["churn"]
del df_val["churn"]

## **Exploratory Data Analysis (EDA)**

In [14]:
# Checking for missing values in full training dataset
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

No missing values to handle!

In [18]:
# Checking the distribution of values in the target variable
df_train_full["churn"].value_counts()

0    4113
1    1521
Name: churn, dtype: int64

In [37]:
# Estimating churn rate
global_cr = df_train_full["churn"].mean()
print("The churn rate is =", np.round(global_cr, 3))

The churn rate is = 0.27


In [26]:
categorical = [
    "gender", "seniorcitizen", "partner",
    "dependents", "phoneservice", "multiplelines",
    "internetservice", "onlinesecurity", "onlinebackup",
    "deviceprotection", "techsupport", "streamingtv",
    "streamingmovies", "contract", "paperlessbilling",
    "paymentmethod"
]

numerical = ["tenure", "monthlycharges", "totalcharges"]

In [27]:
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

### **Feature Importance**

In [39]:
# Churn rate by gender
female_cr = df_train_full[df_train_full["gender"] == "female"]["churn"].mean()
male_cr = df_train_full[df_train_full["gender"] == "male"]["churn"].mean()
print("Global churn rate: ", np.round(global_cr, 4))
print("Female churn rate: ", np.round(female_cr, 4))
print("Male churn rate: ", np.round(male_cr, 4))

Global churn rate:  0.27
Female churn rate:  0.2768
Male churn rate:  0.2632


There's no much difference between them

In [44]:
# Churn rate by having partner
partner_yes_cr = df_train_full[df_train_full["partner"] == "yes"]["churn"].mean()
partner_no_cr = df_train_full[df_train_full["partner"] == "no"]["churn"].mean()
print("Global churn rate: ", np.round(global_cr, 4))
print("partner == yes", np.round(partner_yes_cr, 4))
print("partner == no", np.round(partner_no_cr, 4))

Global churn rate:  0.27
partner == yes 0.205
partner == no 0.3298


The churn rate for people with a partner is considerably less than the rate for the ones without a partner which indicates that the `partner` variable is usefull for predicting churn.

Another interesting way to look for the posibility of customers to churn is to calculate the risk ratio. If the risk ratio is lower than 1, the group has a lower risk of churning. If the risk is higher than 1, the group has a larger risk of churning. If it's close to one, the risk is almost none.

In [50]:
# Calculating the risk of churning (Risk ratio = Group rate / Global rate)
df_group = df_train_full.groupby("gender")["churn"].agg(["mean"])
df_group["diff"] = df_group["mean"] - global_cr
df_group["risk"] = df_group["mean"] / global_cr
df_group

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


In [52]:
# For al categorical variables
for col in categorical:
    df_group = df_train_full.groupby(by=col)["churn"].agg(["mean"])
    df_group["diff"] = df_group["mean"] - global_cr
    df_group["risk"] = df_group["mean"] / global_cr
    display(df_group)

,mean,diff,risk
gender,,,
female,0.276824,0.006856,1.025396
male,0.263214,-0.006755,0.974980


,mean,diff,risk
seniorcitizen,,,
0,0.242270,-0.027698,0.897403
1,0.413377,0.143409,1.531208


,mean,diff,risk
partner,,,
no,0.329809,0.059841,1.221659
yes,0.205033,-0.064935,0.759472


,mean,diff,risk
dependents,,,
no,0.313760,0.043792,1.162212
yes,0.165666,-0.104302,0.613651


,mean,diff,risk
phoneservice,,,
no,0.241316,-0.028652,0.893870
yes,0.273049,0.003081,1.011412


...

,mean,diff,risk
streamingtv,,,
no,0.342832,0.072864,1.269897
no_internet_service,0.077805,-0.192163,0.288201
yes,0.302723,0.032755,1.121328


,mean,diff,risk
streamingmovies,,,
no,0.338906,0.068938,1.255358
no_internet_service,0.077805,-0.192163,0.288201
yes,0.307273,0.037305,1.138182


,mean,diff,risk
contract,,,
month-to-month,0.431701,0.161733,1.599082
one_year,0.120573,-0.149395,0.446621
two_year,0.028274,-0.241694,0.104730


,mean,diff,risk
paperlessbilling,,,
no,0.172071,-0.097897,0.637375
yes,0.338151,0.068183,1.252560


,mean,diff,risk
paymentmethod,,,
bank_transfer_(automatic),0.168171,-0.101797,0.622928
credit_card_(automatic),0.164339,-0.105630,0.608733
electronic_check,0.455890,0.185922,1.688682
mailed_check,0.193870,-0.076098,0.718121
